In [17]:
import os
ipynb_path = os.getcwd()
src_path = os.path.join(ipynb_path, '../src/')
input_path = os.path.join("../input/")

In [18]:
!pip install windpowerlib

In [20]:

import warnings
import os
import sys

import scipy.stats as spst

sys.path.append(src_path)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask
import dask.dataframe as dd
from windpowerlib.wind_speed import logarithmic_profile
from utils import uv_to_wsd

In [21]:
os.getcwd()

'/home/user/Wind_Power/notebooks'

In [22]:
power_2020 = pd.read_parquet(input_path + "dynamic_report_ewp02_2020_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power_2021 = pd.read_parquet(input_path + "dynamic_report_ewp02_2021_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power_2022 = pd.read_parquet(input_path + "dynamic_report_ewp02_2022_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power = pd.concat([power_2020, power_2021, power_2022], ignore_index=True)

gj_y = pd.read_parquet(input_path + "train_y.parquet").rename({'end_datetime': 'dt'}, axis=1)
ldaps = pd.read_parquet(input_path + "train_ldaps_gyeongju.parquet")

print("Power: ", power.shape)
print("train_y: ", gj_y.shape)
print("LDAPS: ", ldaps.shape)

Power:  (155528, 29)
train_y:  (52608, 4)
LDAPS:  (235818, 15)


In [23]:
datas = [power, gj_y, ldaps]
for d in datas:
    try:
        d['dt'] = (pd.to_datetime(d['dt'])
                    .dt
                    .tz_convert("Asia/Seoul"))
    except TypeError:
        d['dt'] = (pd.to_datetime(d['dt'])
                    .dt
                    .tz_localize("Asia/Seoul"))

gj_y = (gj_y.loc[(gj_y['plant_name'] == "경주풍력")
                 & (gj_y['dt']).between('2020-01-01', '2022-12-31', inclusive='left')])

ldaps = ldaps.loc[ldaps['dt'].between('2020-01-01', '2022-12-31', inclusive='left')]

print("Power: ", power.shape, power['dt'].min(), power['dt'].max())
print("train_y: ", gj_y.shape, gj_y['dt'].min(), gj_y['dt'].max())
print("LDAPS: ", ldaps.shape, ldaps['dt'].min(), ldaps['dt'].max())

Power:  (155528, 29) 2020-01-01 00:00:00+09:00 2022-12-31 23:50:00+09:00
train_y:  (26279, 4) 2020-01-01 01:00:00+09:00 2022-12-30 23:00:00+09:00
LDAPS:  (235602, 15) 2020-01-02 00:00:00+09:00 2022-12-30 23:00:00+09:00


In [24]:
# add wind speed in 100m feature
ldaps["wind_speed"], ldaps["wind_direction"] = uv_to_wsd(
    ldaps["wind_u_10m"], ldaps["wind_v_10m"]
)

ldaps["wind_speed_100m"] = logarithmic_profile(ldaps["wind_speed"], 10, 100, 0.3)

/tmp/ipykernel_104481/85188720.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldaps["wind_speed"], ldaps["wind_direction"] = uv_to_wsd(
/tmp/ipykernel_104481/85188720.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldaps["wind_speed_100m"] = logarithmic_profile(ldaps["wind_speed"], 10, 100, 0.3)


In [28]:
print(ldaps.columns)
print(power.columns)

Index(['dt', 'elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m',
       'frictional_vmin_50m', 'pressure', 'relative_humid', 'specific_humid',
       'temp_air', 'storm_u_5m', 'storm_v_5m', 'wind_u_10m', 'wind_v_10m',
       'turbine_id', 'wind_speed', 'wind_direction', 'wind_speed_100m'],
      dtype='object')
Index(['dt', 'turbine_id', 'WTG.Serial', 'AvailabilityForcedOutageTime[Min.]',
       'AvailabilityFullPerformanceTime[Min.]',
       'AvailabilityRequestedShutdownTime[Min.]',
       'AvailabilityScheduledMaintenanceTime[Min.]',
       'AvailabilityTechnicalStandbyTime[Min.]',
       'EnergyProductionActiveEnergyProduction[KWh]',
       'GeneratorAverageWindingTemp.[℃]', 'GridActivePower[kW]',
       'GridReactivePower[kVAr]', 'HydraulicSystemPressure[bar]',
       'NacelleAirDensity[kg/㎥]', 'NacelleNacellePosition[deg]',
       'NacelleOutdoorTemp[℃]', 'NacelleWindDirection[deg]',
       'NacelleWindSpeed[m/s]', 'RotorBlade1Pos.[deg]', 'RotorBlade2Pos.[deg]',
     

In [31]:
ldaps['turbine_id'].value_counts()

WTG01    26178
WTG02    26178
WTG03    26178
WTG04    26178
WTG05    26178
WTG06    26178
WTG07    26178
WTG08    26178
WTG09    26178
Name: turbine_id, dtype: int64

In [ ]:
wtg_df = pd.merge(ldaps, power, on='turbine_id', how='inner') 